In [2]:
import numpy as np 
import pandas as pd 
from subprocess import check_output
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

/Users/westspringamerica/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
max_features = 20000
maxlen = 100


In [4]:
train = pd.read_csv('../downloads/big data/train_NBSCV.csv')
test = pd.read_csv('../downloads/big data/test_NBSCV.csv')
subm = pd.read_csv('../downloads/big data/subm_NBSCV.csv')

In [8]:
train = train.sample(frac=1)

list_sentences_train = train["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").values


tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)


In [48]:
y_test

array([[0.9854835 , 0.1555516 , 0.9098244 , 0.00806442, 0.5128318 ,
        0.0393456 ]], dtype=float32)

In [29]:
examplesen

array(['fuck off'], dtype=object)

In [18]:
def def_model():
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = LSTM(75, return_sequences=True)(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.15)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    #x = Dropout(0.2)(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model


model = def_model()
batch_size = 32
epochs = 2


file_path="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor="val_loss", mode="min", patience=20)


callbacks_list = [checkpoint, early] #early
model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)

model.load_weights(file_path)

y_test = model.predict(X_te)

Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 645s 4ms/step - loss: 0.0709 - acc: 0.9772 - val_loss: 0.0530 - val_acc: 0.9812

Epoch 00001: val_loss improved from inf to 0.05302, saving model to weights_base.best.hdf5
Epoch 2/3
143613/143613 [==============================] - 14168s 99ms/step - loss: 0.0466 - acc: 0.9828 - val_loss: 0.0500 - val_acc: 0.9819

Epoch 00002: val_loss improved from 0.05302 to 0.05000, saving model to weights_base.best.hdf5
Epoch 3/3
143613/143613 [==============================] - 608s 4ms/step - loss: 0.0408 - acc: 0.9842 - val_loss: 0.0503 - val_acc: 0.9814

Epoch 00003: val_loss did not improve from 0.05000


In [ ]:
model.load_weights(file_path)

y_test = model.predict(X_te)

In [16]:
#print(model.predict(np.array[['fuck off']]))

TypeError: 'builtin_function_or_method' object is not subscriptable

In [57]:
#list_tokenized_test
#y = train[list_classes].values
list_sentences_test
examplesen=np.array(["shit"],dtype=object)
examplestest = tokenizer.texts_to_sequences(examplesen)
X_test=sequence.pad_sequences(examplestest, maxlen=maxlen)
y_test = model.predict(X_test)
y_test

array([[0.9854835 , 0.1555516 , 0.9098244 , 0.00806442, 0.5128318 ,
        0.0393456 ]], dtype=float32)

In [1]:
#model.savemodel("lstm.model")

In [54]:
#model.save("../downloads/big data/lstm.model")